# Fine tune MiniChat-1.5-3B model using Axolotl framework

How to install dependencies (in HPC environment):

- load Python and cuDNN modules
- create a Python venv and activate it
- install dependencies from requirements.txt (e.g. torch)
- install Axolotl from git clone (pip won't work, see [this issue](https://github.com/OpenAccess-AI-Collective/axolotl/issues/945)):

```
git clone git@github.com:OpenAccess-AI-Collective/axolotl.git
cd axolotl
pip install -e '.[flash-attn,deepspeed]'
```


In [1]:
# Check if GPU is available
import torch
print('GPU available?', torch.cuda.is_available())
print('BF16 is supported?', torch.cuda.is_bf16_supported())

GPU available? True
BF16 is supported? True


In [2]:
# set model name etc.

MODEL_NAME = "GeneZC/MiniChat-1.5-3B"
MODEL_SHORT_NAME = MODEL_NAME.split('/')[-1]

In [3]:
# Load and prepare fine-tuning dataset

import json
import glob
import random

random.seed(42)  # for deterministic sampling of test set

train_files = glob.glob("../../llm-dataset/*-train.jsonl")
test_files = glob.glob("../../llm-dataset/*-test.jsonl")

KEEP_FIELDS = {
    'dc.contributor.author',
    'dc.date.issued',
    'dc.identifier.isbn',
    'dc.language.iso',
    'dc.publisher',
    'dc.relation.eissn',
    'dc.title'    
}
MAX_TEXT_LENGTH = 3072
EVAL_SIZE = 32  # how many documents to evaluate (i.e. calculate loss) on during fine-tuning
INSTRUCTION = "Extract metadata. Return as JSON."

def preprocess_sample(sample):
    # subset & JSON encode the ground truth
    subset = {key: val
              for key, val in sample["ground_truth"].items()
              if key in KEEP_FIELDS}
    output = json.dumps(subset)
    input_ = sample["text"][:MAX_TEXT_LENGTH]
    # Alpaca format
    return {"instruction": INSTRUCTION, "input": input_, "output": output}

def dataset_to_records(files):
    records = []
    for filename in files:
        with open(filename) as infile:
            for line in infile:
                sample = json.loads(line)
                records.append(preprocess_sample(sample))
    return records

def write_jsonl(records, filename):
    with open(filename, "w") as outfile:
        for record in records:
            json.dump(record, outfile)
            outfile.write("\n")

train_recs = dataset_to_records(train_files)
random.shuffle(train_recs)
write_jsonl(train_recs, "train.jsonl")
print(f"Wrote {len(train_recs)} train records")

test_recs = dataset_to_records(test_files)
write_jsonl(test_recs, "test.jsonl")
print(f"Wrote {len(test_recs)} test records")

eval_recs = random.sample(test_recs, EVAL_SIZE)
write_jsonl(eval_recs, "eval.jsonl")
print(f"Wrote {len(eval_recs)} eval records")

Wrote 556 train records
Wrote 167 test records
Wrote 32 eval records


In [4]:
# Create Axolotl configuration file

CONFIG_FILE = f"config-{MODEL_SHORT_NAME}.yml"


CONFIG = f"""
base_model: {MODEL_NAME}
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
is_llama_derived_model: true

load_in_8bit: true
load_in_4bit: false
strict: false

datasets:
  - path: train.jsonl
    ds_type: json
    split: train
    type: alpaca

test_datasets:
  - path: eval.jsonl
    ds_type: json
    split: train
    type: alpaca

output_dir: ./out-{MODEL_SHORT_NAME}

adapter: lora
lora_r: 16
lora_alpha: 32
lora_dropout: 0.05
lora_target_modules:
  - q_proj
  - v_proj
  - k_proj
  - o_proj
  - gate_proj
  - down_proj
  - up_proj
sequence_len: 4096
sample_packing: true
eval_sample_packing: false
pad_to_sequence_len: true

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 4
micro_batch_size: 4
eval_batch_size: 4
num_epochs: 5
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: false

gradient_checkpointing: true  # true: saves VRAM but is slower to train
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

warmup_steps: 10
evals_per_epoch: 2
eval_table_size:
eval_table_max_new_tokens: 128
saves_per_epoch: 1
debug:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
  bos_token: "<s>"
  eos_token: "</s>"
  unk_token: "<unk>"
  pad_token: "<unk>"
""".strip()

with open(CONFIG_FILE, 'w') as outfile:
    print(CONFIG, file=outfile)

In [5]:
%%time

!venv/bin/accelerate launch -m axolotl.cli.train {CONFIG_FILE}

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2024-02-16 15:10:43,815] [INFO] [datasets.<module>:58] [PID:2883644] PyTorch version 2.2.0 available.
[2024-02-16 15:10:47,518] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-02-16 15:10:51,327] [INFO] [axolotl.normalize_config:170] [PID:2883644] [RANK:0] GPU memory usage baseline: 0.000GB (+0.818GB misc)
                                 dP            dP   dP 
                                 88            88   88 
      .d8888b. dP.  .dP .d8888b. 88 .d8888b. d8888P 88 
      88'  `88  `8bd8'  88'  `88 88 88'  `88   88   88 
      88.  .88  .d88b.  88.  .88

# Use the fine-tuned model

In [10]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

qlora_model = f"./out-{MODEL_SHORT_NAME}"
base_model = MODEL_NAME
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", torch_dtype=torch.float16, attn_implementation="flash_attention_2").eval()
model = PeftModel.from_pretrained(model, qlora_model)
model = model.merge_and_unload()



In [7]:
def generate(prompt):
    input_ids = tokenizer([prompt]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        max_new_tokens=512,
    )
    output_ids = output_ids[0][len(input_ids[0]):]
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()


In [8]:
%%time

import json

PROMPT_TEMPLATE = """
### Instruction:
{instruction}

### Input:
{input}

### Response:
""".strip()

with open(f'test-records-{MODEL_SHORT_NAME}.jsonl', 'w') as outfile:
    for rec in test_recs:
        prompt = PROMPT_TEMPLATE.format(instruction=INSTRUCTION, input=rec['input'])
        response = generate(prompt)

        print(100 * "-")
        print("Ground Truth:")
        print(rec['output'])
        print("Prediction:")
        print(response)
        
        ground_truth = json.loads(rec['output'])
        try:
            prediction = json.loads(response)
        except json.JSONDecodeError:
            prediction = {}
        
        # rowid is set to unknown as we've lost it somewhere along the way...
        record = {"ground_truth": ground_truth, "prediction": prediction, "rowid": "unknown"}
        json.dump(record, outfile)
        outfile.write("\n")

----------------------------------------------------------------------------------------------------
Ground Truth:
{"dc.title": "Poliisikoulutuksen vaikuttavuusarviointi 2021 : vuosina 2018-2019 valmistuneiden poliisien ty\u00f6llisyys ja arviot koulutuksen ty\u00f6el\u00e4m\u00e4vastaavuudesta", "dc.contributor.author": ["Vuorensyrj\u00e4, Matti"], "dc.date.issued": "2021", "dc.identifier.isbn": ["978-951-815-386-6"], "dc.language.iso": "fin", "dc.publisher": ["Poliisiammattikorkeakoulu"]}
Prediction:
{"dc.title": "Poliisikoulutuksen vaikuttavuusarviointi 2021", "dc.contributor.author": ["Vuorensyrj\u00e4, Matti"], "dc.date.issued": "2021-05-28", "dc.identifier.isbn": ["978-951-815-386-6"], "dc.language.iso": "fin", "dc.publisher": ["Grano Oy"]}
----------------------------------------------------------------------------------------------------
Ground Truth:
{"dc.title": "FAQ : Taiteen digitaaliset toimintaymp\u00e4rist\u00f6t", "dc.date.issued": "2018", "dc.identifier.isbn": ["978-95

In [9]:
# Analyze the statistics of the extracted metadata and save to file

import sys
sys.path.append('..')

from eval import MetadataEvaluator

evaluator = MetadataEvaluator(f'test-records-{MODEL_SHORT_NAME}.jsonl')
results = evaluator.evaluate_records() #prediction_records[:9])
# Use only the fields that Meteor extracts
fields = [
        "dc.contributor.author",
        "dc.date.issued",
        "dc.identifier.isbn",
        "dc.language.iso",
        "dc.publisher",
        "dc.relation.eissn",
        "dc.title",
    ]
statistics_filename = '../results-axolotl-fine-tune-' + MODEL_SHORT_NAME + '.md'
evaluator.save_md(results, statistics_filename, fields)